In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install git+https://github.com/facebookresearch/fvcore.git
!pip install pytorchvideo
!pip install transformers


  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-s_tdde52
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-s_tdde52
  Resolved https://github.com/facebookresearch/fvcore.git to commit 0f2b23b6f93e36041d9a74764ee824541cf0a0e5
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.6-py3-none-any.whl size=65266 sha256=23cbfd26d0837ab9f0160c1eca451879385c1515ab584b3f42d0fde6eef10865
  Stored in directory: /tmp/pip-ephem-wheel-cache-jp7u__03/wheels/8f/cb/6a/3b7ac0e01781855ca3d1417ebf9e15e20d5b7fe37ab063aa50
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31530 sha256=d0c96fb77ddd0b5908fb63c018be845f808eaf9c49c2f7c71f7bd66dd5325fc6
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e47

In [3]:
import random
import pandas as pd
import numpy as np
import os
import cv2
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torch.backends.cudnn as cudnn

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models
import torchvision
import torch.nn as nn
from sklearn.metrics import f1_score,accuracy_score

from transformers.optimization import AdamW


from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42) # Seed 고정
from pytorchvideo.transforms import Normalize

# CFG = {
#     "VIDEO_LENGTH":
# }



In [4]:
train = pd.read_csv("/content/drive/MyDrive/data/train_class9.csv",index_col=0)
valid = pd.read_csv("/content/drive/MyDrive/data/valid_class9.csv",index_col=0)
test = pd.read_csv("/content/drive/MyDrive/data/test_class9.csv",index_col=0)

In [5]:
train = train[~train['video_name'].str.contains('Store')]
valid = valid[~valid['video_name'].str.contains('Store')]
test = test[~test['video_name'].str.contains('Store')]

In [6]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [7]:
mean = (0.45, 0.45, 0.45)
std = (0.225,0.225, 0.225)

class CustomDataset(Dataset):
  def __init__(self, video_path_list,label_list):
    self.video_path_list = video_path_list
    self.label_list = label_list
    self.norm = Normalize(mean, std)

  def __getitem__(self, index):
    try:
      frames = self.get_video(self.video_path_list[index])[:,2:,:,:]
      low_frames = frames[:,[i*4 for i in range(19)],:,:] #19

      if self.label_list is not None:
        label = self.label_list[index]
        return low_frames, frames, label #프레임 적고/크고
      else:
        return low_frames, frames #slow, fast #이 부분을 없애야 하나.

    except Exception as e:
      print(f"An exception occurred in get_video function: {e}")
      return None


  def __len__(self):
    return len(self.video_path_list)

  def get_video(self, path):
    frames = []
    cap = cv2.VideoCapture(path)
    for _ in range(77): #77
      _, img = cap.read()

      if img is not None:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (256,256))
        img = img/255
        frames.append(img)

    cap.release()

    return self.norm(torch.FloatTensor(np.array(frames)).permute(3,0,1,2))

def aug_video(vid, tfms):
    aug_vid = []
    for x in vid:
        aug_vid.append((tfms(image = np.asarray(x)))['image'])
    return torch.from_numpy(np.stack(aug_vid))

tfms = A.Compose([
            A.HorizontalFlip(p=0.5),
             A.GaussNoise(always_apply=False, p=0.5, var_limit=(50.00, 100.00), per_channel=True, mean=0.0)
            ], p=1)


In [8]:
train.head()

,video_name,tag,class_0,class_1,class_2,class_3,class_4,class_5,class_6,class_7,class_8
471,/content/drive/MyDrive/data/normal/시험/C_2_3...,3,0,0,0,1,0,0,0,0,0
1453,/content/drive/MyDrive/data/abnormal/폭행/C_...,7,0,0,0,0,0,0,0,1,0
2379,/content/drive/MyDrive/data/abnormal/절도/C_3...,8,0,0,0,0,0,0,0,0,1
1601,/content/drive/MyDrive/data/abnormal/폭행/C_...,7,0,0,0,0,0,0,0,1,0
1094,/content/drive/MyDrive/data/abnormal/파손/C_3...,6,0,0,0,0,0,0,1,0,0


In [9]:
train_dataset = CustomDataset(train["video_name"].values, train["tag"].values)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)

val_dataset = CustomDataset(valid["video_name"].values, valid["tag"].values)
val_loader = DataLoader(val_dataset, batch_size =4, shuffle=False, num_workers=0)

In [11]:
backbone = torch.hub.load("facebookresearch/pytorchvideo", model="slowfast_r50",pretrained=True)


Downloading: "https://github.com/facebookresearch/pytorchvideo/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/pytorchvideo/model_zoo/kinetics/SLOWFAST_8x8_R50.pyth" to /root/.cache/torch/hub/checkpoints/SLOWFAST_8x8_R50.pyth
100%|██████████| 264M/264M [00:01<00:00, 188MB/s]


In [ ]:
# class Slowfast(nn.Module):
#   def __init__(self, num_classes=8, model_name = "slowfast_r50"):
#     super(Slowfast,self).__init__()
#     self.backbone = torch.hub.load("facebookresearch/pytorchvideo", model=model_name,pretrained=True)

#     output_dim = 400 #slowfast의 마지막 출력값.

#     self.classifier = nn.Sequential(
#         nn.Linear(output_dim, 256),
#         nn.ReLU(),
#         nn.Linear(256, num_classes)
#     )

#   def forward(self, x):
#     print("x1 : ",x[0].shape)
#     print("x2 : ",x[1].shape)
#     x = self.backbone(x)
#     print("x3 : ",x[0].shape)

#     x = self.classifier(x)

#     return x

In [12]:
model_name = "slowfast_r50"
slowfast_r50 = torch.hub.load("facebookresearch/pytorchvideo", model=model_name,pretrained=True)
slowfast_r50_layers = slowfast_r50.blocks

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [ ]:
# class SlowFast(nn.Module):
#     def __init__(self,img_size=256):
#         super(SlowFast, self).__init__()
#         self.pool_size = img_size//32-1

#         self.layer_1 = slowfast_r50_layers[0]
#         self.layer_2 = slowfast_r50_layers[1]
#         self.layer_3 = slowfast_r50_layers[2]
#         self.layer_4 = slowfast_r50_layers[3]
#         self.layer_5 = slowfast_r50_layers[4]
#         self.fast_pool = nn.AvgPool3d(kernel_size=(19, self.pool_size, self.pool_size), stride=(1, 1, 1), padding=(0, 0, 0))
#         self.slow_pool = nn.AvgPool3d(kernel_size=(75, self.pool_size, self.pool_size), stride=(1, 1, 1), padding=(0, 0, 0))

#         self.avgpool = nn.AdaptiveAvgPool3d(1)
#         self.fc = nn.Linear(2304,4)

#         for para in self.layer_1.parameters():
#            para.requires_grad = False
#         for para in self.layer_2.parameters():
#            para.requires_grad = False

#     def forward(self, x):
#         # print("flag1")
#         batch_size = x[0].size(0)
#         # print("flag2")

#         x = self.layer_1(x)
#         # print("flag3")

#         x = self.layer_2(x)
#         # print("flag4")

#         x = self.layer_3(x)
#         # print("flag5")

#         x = self.layer_4(x)
#         # print("flag6")

#         x = self.layer_5(x)
#         # print("flag7")

#         # print("x[0] : ",x[0].shape)
#         # print("x[1] : ",x[1].shape)



#         x_1 = self.fast_pool(x[0])
#         #print("flag8")

#         x_2 = self.slow_pool(x[1])
#         #print("flag9")
#         # print("x_1 : ", x_1.shape)
#         # print("x_2 : ", x_2.shape)

#         x = torch.concat([x_1,x_2],axis=1)
#         #print("flag10")

#         x = self.avgpool(x).view(batch_size,-1)
#         #print("flag11")


#         x = self.fc(x)

#         return x

In [13]:
class SlowFast(nn.Module):
    def __init__(self,img_size=256):
        super(SlowFast, self).__init__()
        self.pool_size = img_size//32-1

        self.layer_1 = slowfast_r50_layers[0]
        self.layer_2 = slowfast_r50_layers[1]
        self.layer_3 = slowfast_r50_layers[2]
        self.layer_4 = slowfast_r50_layers[3]
        self.layer_5 = slowfast_r50_layers[4]
        self.fast_pool = nn.AvgPool3d(kernel_size=(19, self.pool_size, self.pool_size), stride=(1, 1, 1), padding=(0, 0, 0))
        self.slow_pool = nn.AvgPool3d(kernel_size=(75, self.pool_size, self.pool_size), stride=(1, 1, 1), padding=(0, 0, 0))

        self.avgpool = nn.AdaptiveAvgPool3d(1)
        self.fc = nn.Linear(2304,9)

        for para in self.layer_1.parameters():
           para.requires_grad = False
        for para in self.layer_2.parameters():
           para.requires_grad = False

    def forward(self, x):
        # print("flag1")
        batch_size = x[0].size(0)
        # print("flag2")

        x = self.layer_1(x)
        nn.Dropout(0.3)
        # print("flag3")

        x = self.layer_2(x)
        nn.Dropout(0.3)
        # print("flag4")

        x = self.layer_3(x)
        nn.Dropout(0.3)
        # print("flag5")

        x = self.layer_4(x)
        nn.Dropout(0.3)
        # print("flag6")

        x = self.layer_5(x)
        nn.Dropout(0.3)
        # print("flag7")

        # print("x[0] : ",x[0].shape)
        # print("x[1] : ",x[1].shape)



        x_1 = self.fast_pool(x[0])
        #print("flag8")

        x_2 = self.slow_pool(x[1])
        #print("flag9")
        # print("x_1 : ", x_1.shape)
        # print("x_2 : ", x_2.shape)

        x = torch.concat([x_1,x_2],axis=1)
        #print("flag10")
        nn.Dropout(0.3)

        x = self.avgpool(x).view(batch_size,-1)
        nn.Dropout(0.3),
        #print("flag11")


        x = self.fc(x)

        return x

In [14]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [15]:
# torch.cuda.empty_cache()
file_name = 'submit'

model = SlowFast(img_size=256)
model.eval()
best_model = None
optimizer = optim.AdamW(params = model.parameters(), lr = 0.0001,weight_decay=0.02)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[12], gamma=0.1)
model = model.to(device)

In [ ]:
# check = 0
# i = 0
# best_val_score = 0
# criterion = nn.CrossEntropyLoss().to(device)


# train_loss = []

# for epoch in range(1, 10+1):
#     model.train()
#     train_loss = []
#     output_preds = []
#     output_trues = []
#     for low_videos, videos, labels in tqdm(iter(train_loader)):
#         if low_videos is None or videos is None or labels is None:
#           continue
#         i+=1
#         low_videos = low_videos.to(device)

#         videos = videos.to(device)
#         labels = labels.to(device)

#         optimizer.zero_grad()

#         output= model([low_videos,videos])
#         loss = criterion(output,labels)

#         loss.backward()
#         optimizer.step()

#         train_loss.append(loss.item())

#         output_preds += output.argmax(1).detach().cpu().numpy().tolist()
#         output_trues += labels.detach().cpu().numpy().tolist()

#     scheduler.step()
#     _train_loss = np.mean(train_loss)
#     train_score = f1_score(output_trues, output_preds, average="macro")
#     train_accuracy = accuracy_score(output_trues, output_preds)




#     print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Train Acc : [{train_accuracy:.5f}]] Train F1 : [{train_score :.5f}]\n')


In [16]:
def validation(model, criterion, val_loader, device):
  model.eval()
  val_loss = []
  preds, trues = [],[]

  with torch.no_grad():
    # for videos, labels in tqdm(iter(val_loader)):
    for slow_videos, videos, labels in tqdm(iter(val_loader)):
      slow_videos = slow_videos.to(device)
      videos = videos.to(device)
      labels = labels.to(device)
      print(labels)

      logit = model([slow_videos,videos])

      loss = criterion(logit, labels)
      val_loss.append(loss.item())

      preds +=logit.argmax(1).detach().cpu().numpy().tolist()
      trues += labels.detach().cpu().numpy().tolist()

    _val_loss = np.mean(val_loss)

  _val_score = f1_score(trues, preds, average="macro")
  return _val_loss, _val_score

In [18]:
def train_(model, optimizer, train_loader, val_loader, schedule, device):
  model.to(device)
  criterion = nn.CrossEntropyLoss().to(device)

  best_val_score = 0
  best_model = None

  for epoch in range(1,1+1): #테스트 용이니까.
    model.train()
    train_loss=[]
    for slow_videos, videos, labels in tqdm(train_loader):
      slow_videos = slow_videos.to(device)
      videos = videos.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()
      output = model([slow_videos,videos])

      loss = criterion(output, labels)

      loss.backward()
      optimizer.step()

      train_loss.append(loss.item())

    _val_loss, _val_score = validation(model, criterion, val_loader, device)
    _train_loss = np.mean(train_loss)
    print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]')

    if schedule is not None:
      schedule.step(_val_score)

    if best_val_score < _val_score:
      best_val_score = _val_score
      best_model = model

  return best_model

In [19]:
model.eval()
# optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
optimizer = torch.optim.AdamW(params = model.parameters(), lr =0.001, weight_decay=0.05)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[12], gamma=0.1)

# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=0.5, patience=2, threshold_mode="abs",
                                                      # min_lr=1e-8, verbose=True)
infer_model = train_(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/577 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

tensor([5, 7, 8, 5], device='cuda:0')
tensor([8, 6, 6, 1], device='cuda:0')
tensor([4, 6, 7, 6], device='cuda:0')
tensor([6, 4, 8, 2], device='cuda:0')
tensor([7, 4, 6, 8], device='cuda:0')
tensor([6, 8, 8, 8], device='cuda:0')
tensor([6, 4, 6, 8], device='cuda:0')
tensor([0, 7, 7, 7], device='cuda:0')
tensor([7, 8, 6, 6], device='cuda:0')
tensor([8, 0, 4, 0], device='cuda:0')
tensor([8, 4, 7, 8], device='cuda:0')
tensor([6, 8, 8, 8], device='cuda:0')
tensor([6, 8, 8, 3], device='cuda:0')
tensor([6, 3, 8, 1], device='cuda:0')
tensor([8, 6, 6, 7], device='cuda:0')
tensor([8, 7, 1, 7], device='cuda:0')
tensor([8, 3, 7, 5], device='cuda:0')
tensor([8, 3, 4, 6], device='cuda:0')
tensor([8, 0, 5, 6], device='cuda:0')
tensor([7, 8, 6, 8], device='cuda:0')
tensor([8, 8, 6, 7], device='cuda:0')
tensor([6, 6, 6, 6], device='cuda:0')
tensor([0, 6, 6, 8], device='cuda:0')
tensor([4, 7, 7, 6], device='cuda:0')
tensor([5, 8, 6, 8], device='cuda:0')
tensor([6, 8, 6, 7], device='cuda:0')
tensor([3, 7

In [20]:
path = "/content/drive/MyDrive/models"


# 모델 상태를 해당 경로에 저장합니다.
file_path = os.path.join(path, f"{file_name}_last_epoch_model.pt")
torch.save(model.state_dict(), file_path)
print(f"Last epoch model saved successfully at {file_path}.")

Last epoch model saved successfully at /content/drive/MyDrive/models/submit_last_epoch_model.pt.


In [23]:
from sklearn.metrics import confusion_matrix
def test_(model, criterion, test_loader, device):
  model.eval()
  test_loss = []
  preds, trues = [],[]

  with torch.no_grad(): #back prop~~ 안하기.
    # for videos, labels in tqdm(iter(val_loader)):
    for slow_videos, videos, labels in tqdm(iter(test_loader)):
      slow_videos = slow_videos.to(device)
      videos = videos.to(device)
      labels = labels.to(device)
      print(labels)

      logit = model([slow_videos,videos])

      loss = criterion(logit, labels)
      test_loss.append(loss.item())

      preds +=logit.argmax(1).detach().cpu().numpy().tolist()
      trues += labels.detach().cpu().numpy().tolist()

    _test_loss = np.mean(test_loss)

  _test_score = f1_score(trues, preds, average="macro")

  confusion_mat = confusion_matrix(trues, preds)

    # Calculate label-wise accuracy
  label_accuracies = []
  for i in range(confusion_mat.shape[0]):
      label_accuracy = confusion_mat[i, i] / confusion_mat[i, :].sum()
      label_accuracies.append(label_accuracy)
  return _test_loss, _test_score,  label_accuracies, confusion_mat

In [22]:
tq = pd.read_csv("/content/drive/MyDrive/data/test_equal.csv")
tq = tq[~tq['video_name'].str.contains('Store')]

In [ ]:
test = pd.read_csv("/content/drive/MyDrive/data/test_class9.csv")
test_kaggle = pd.read_csv("/content/drive/MyDrive/data/test_kaggle.csv")
test_samplevids = pd.read_csv('/content/drive/MyDrive/samplevids/test_samplevids.csv')

In [24]:
test_dataset = CustomDataset(tq["video_name"].values, tq["tag"].values)
test_loader = DataLoader(test_dataset, batch_size =4, shuffle=False, num_workers=0)

# t1 = pd.read_csv("/content/drive/MyDrive/data/test_kaggle.csv")
# t1_dataset = CustomDataset(t1["video_name"].values, t1["tag"].values)
# t1_loader = DataLoader(t1_dataset, batch_size =4, shuffle=False, num_workers=0)

model = SlowFast(img_size=256)
model.load_state_dict(torch.load("/content/drive/MyDrive/models/submit_last_epoch_model.pt"))
model.to(device)


SlowFast(
  (layer_1): MultiPathWayWithFuse(
    (multipathway_blocks): ModuleList(
      (0): ResNetBasicStem(
        (conv): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
        (norm): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
        (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
      )
      (1): ResNetBasicStem(
        (conv): Conv3d(3, 8, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False)
        (norm): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
        (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
      )
    )
    (multipathway_fusion): FuseFastToSlow(
      (conv_fast_to_slow): Conv3d(8, 16, kernel_size=(7, 1, 1), stride=(4, 1, 1), padding=(3, 0, 0), bias=False)


In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
_test_loss, _test_score,  label_accuracies, confusion_mat = test_(model, criterion, test_loader, device)

  0%|          | 0/158 [00:00<?, ?it/s]

tensor([0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1

In [ ]:
print("Loss 및 f1 score 보기")
print(f'Test Loss : [{_test_loss:.5f}] test F1 : [{_test_score:.5f}]')

print("라벨의 정확성 보기")
print("label_accuracies : ", label_accuracies)

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

# 예시 Confusion Matrix

# 클래스 레이블
class_labels = ['buy', 'select', 'return', 'test', 'move', 'compare', 'break', 'violence', 'theft']
#class_labels = ['Label 0', 'Label 1', 'Label 2', "Label 3"]
# class_labels = ['Label 3', 'Label 2', 'Label 1', "Label 0"]

# 히트맵 생성
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_mat, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()
